# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [53]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import isnan, when, count, col, min as smin, max as smax

In [2]:
!ls

2m_rows_sparkify_event_data.csv  README.MD
b_1_sparkify_event_data.csv	 requirements.txt
ETL.ipynb			 sparkify_event_data.json
full_sparkify_event_data.json	 Sparkify.ipynb
medium_sparkify_event_data.csv


In [2]:
filepath = '2m_rows_sparkify_event_data.csv'

In [3]:
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

spark.sparkContext.setLogLevel('DEBUG')

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [15]:
df = spark.read.option("inferSchema", "true").option("header", "true").option("encoding", "utf-8").csv(filepath)

In [16]:
df.dtypes

[('status', 'int'),
 ('gender', 'string'),
 ('length', 'double'),
 ('firstName', 'string'),
 ('level', 'string'),
 ('lastName', 'string'),
 ('registration', 'double'),
 ('userId', 'int'),
 ('ts', 'bigint'),
 ('auth', 'string'),
 ('page', 'string'),
 ('sessionId', 'int'),
 ('location', 'string'),
 ('itemInSession', 'int'),
 ('userAgent', 'string'),
 ('song', 'string'),
 ('artist', 'string'),
 ('method', 'string')]

In [17]:
df.show(n=2, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------
 status        | 200                                                                         
 gender        | M                                                                           
 length        | 524.32934                                                                   
 firstName     | Shlok                                                                       
 level         | paid                                                                        
 lastName      | Johnson                                                                     
 registration  | 1.533734541E12                                                              
 userId        | 1749042                                                                     
 ts            | 1538352001000                                                               
 auth          | Logged In                                  

In [18]:
df.columns

['status',
 'gender',
 'length',
 'firstName',
 'level',
 'lastName',
 'registration',
 'userId',
 'ts',
 'auth',
 'page',
 'sessionId',
 'location',
 'itemInSession',
 'userAgent',
 'song',
 'artist',
 'method']

In [19]:
not_na_columns = [ 'userId', 'sessionId' ]

In [20]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+------+------+------+---------+-----+--------+------------+------+---+----+----+---------+--------+-------------+---------+----+------+------+
|status|gender|length|firstName|level|lastName|registration|userId| ts|auth|page|sessionId|location|itemInSession|userAgent|song|artist|method|
+------+------+------+---------+-----+--------+------------+------+---+----+----+---------+--------+-------------+---------+----+------+------+
|     0|     0|     0|        0|    0|       0|           0|     0|  0|   0|   0|        0|       0|            0|        0|   0|     0|     0|
+------+------+------+---------+-----+--------+------------+------+---+----+----+---------+--------+-------------+---------+----+------+------+



In [25]:
df.groupBy('userId').count().orderBy('count', ascending = False).show(50)

+-------+-----+
| userId|count|
+-------+-----+
|1261737|64332|
|1333174| 1931|
|1502571| 1898|
|1140512| 1798|
|1049289| 1768|
|1045674| 1757|
|1748906| 1751|
|1199183| 1713|
|1563125| 1585|
|1564221| 1576|
|1352772| 1545|
|1730425| 1537|
|1967025| 1505|
|1155970| 1504|
|1019103| 1495|
|1169050| 1465|
|1082829| 1459|
|1090699| 1453|
|1204704| 1389|
|1985204| 1378|
|1711691| 1373|
|1434232| 1368|
|1989862| 1365|
|1343610| 1363|
|1342468| 1359|
|1708349| 1352|
|1240184| 1349|
|1006695| 1347|
|1650232| 1339|
|1249037| 1330|
|1116823| 1325|
|1806776| 1323|
|1235208| 1319|
|1200978| 1314|
|1577591| 1310|
|1870948| 1306|
|1422950| 1299|
|1515610| 1298|
|1655121| 1290|
|1662639| 1288|
|1904872| 1274|
|1778160| 1273|
|1395776| 1272|
|1073857| 1270|
|1762462| 1261|
|1156589| 1257|
|1663631| 1257|
|1141500| 1254|
|1987393| 1251|
|1116108| 1250|
+-------+-----+
only showing top 50 rows



In [22]:
df.select("artist").distinct().count()

140616

In [24]:
df.select('length').describe().show()

+-------+------------------+
|summary|            length|
+-------+------------------+
|  count|           1571580|
|   mean|248.82363335965834|
| stddev| 97.59410547771161|
|    min|             0.522|
|    max|        3024.66567|
+-------+------------------+



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [27]:
df.groupBy('page').count().orderBy('count', ascending = False).show(50)

+--------------------+-------+
|                page|  count|
+--------------------+-------+
|            NextSong|1571580|
|                Home| 109774|
|           Thumbs Up|  78871|
|         Roll Advert|  44623|
|     Add to Playlist|  44290|
|          Add Friend|  29403|
|              Logout|  24673|
|               Login|  24094|
|         Thumbs Down|  15897|
|                Help|  12134|
|           Downgrade|  11911|
|            Settings|  11553|
|               About|   7468|
|             Upgrade|   6631|
|       Save Settings|   2321|
|               Error|   1975|
|      Submit Upgrade|   1965|
|    Submit Downgrade|    336|
|              Cancel|    221|
|Cancellation Conf...|    221|
|            Register|     37|
| Submit Registration|     22|
+--------------------+-------+



Some questions about the data:

- Are errors related to downgrading canceling the service?
- Having a certain number of friends or a sense of community can decrease the churn?
- Thumbs down are related to churn? (could the quality of the songs catalog affect the churn)
- The advertising is not annoying the users?
- Users with stay connected for more time have less change to churn?
- Is the home page relevant?
- Users, who access the downgrade page are how much more willing to churn?

In [37]:
df.filter('userId = 1261737').groupBy('page').count().orderBy('count', ascending = False).show(50)

+-------------------+-----+
|               page|count|
+-------------------+-----+
|               Home|34151|
|              Login|24094|
|              About| 3783|
|               Help| 2177|
|              Error|   68|
|           Register|   37|
|Submit Registration|   22|
+-------------------+-----+



In [38]:
df.filter('userId = 1333174').groupBy('page').count().orderBy('count', ascending = False).show(50)

+---------------+-----+
|           page|count|
+---------------+-----+
|       NextSong| 1611|
|           Home|   71|
|      Thumbs Up|   70|
|Add to Playlist|   56|
|      Downgrade|   28|
|     Add Friend|   27|
|         Logout|   19|
|    Thumbs Down|   18|
|           Help|   12|
|       Settings|    7|
|    Roll Advert|    3|
|          About|    2|
|  Save Settings|    2|
|          Error|    2|
|        Upgrade|    2|
| Submit Upgrade|    1|
+---------------+-----+



In [39]:
df.filter('userId = 1333174').groupBy('userAgent').count().orderBy('count', ascending = False).show(50)

+--------------------+-----+
|           userAgent|count|
+--------------------+-----+
|Mozilla/5.0 (Wind...| 1931|
+--------------------+-----+



In [43]:
df.filter('userId = 1333174 and song != \'null\' ').groupBy('song').count().orderBy('count', ascending = False).show(50)

+--------------------+-----+
|                song|count|
+--------------------+-----+
|Dog Days Are Over...|    8|
|                Undo|    7|
|       Sehr kosmisch|    6|
|             Revelry|    6|
|       ReprÃÂ©sente|    5|
|Horn Concerto No....|    5|
|      You're The One|    5|
|           Fireflies|    4|
|            The Gift|    4|
|  Heartbreak Warfare|    4|
| I CAN'T GET STARTED|    4|
|        Use Somebody|    4|
|    Creep (Explicit)|    4|
|    Somebody To Love|    4|
|     Time To Pretend|    3|
|        Make Her Say|    3|
|             Invalid|    3|
|Ironic (Acoustic ...|    3|
|               Gears|    3|
|         Why Georgia|    3|
|    Never Let You Go|    3|
|               I Ran|    3|
|    Hey_ Soul Sister|    3|
|Unwell (Album Ver...|    3|
|              Rianna|    3|
|              Easier|    2|
|         Cosmic Love|    2|
|South Side (Digit...|    2|
|   Hey There Delilah|    2|
|        Wavin'  Flag|    2|
|     Do You Remember|    2|
|I?'m A Steady

In [48]:
df.select(['ts', 'page', 'sessionId', 'itemInSession', 'song', 'artist']).filter('userId = 1333174').orderBy('ts', ascending = False).show(50)

+-------------+---------------+---------+-------------+--------------------+--------------------+
|           ts|           page|sessionId|itemInSession|                song|              artist|
+-------------+---------------+---------+-------------+--------------------+--------------------+
|1538895218000|           Home|    50975|          123|                null|                null|
|1538895186000|       NextSong|    50975|          122|Tonight She Comes...|            The Cars|
|1538895032000|       NextSong|    50975|          121|I?'m A Steady Rol...|      Robert Johnson|
|1538894768000|       NextSong|    50975|          120| Part Of The Process|           Morcheeba|
|1538894456000|       NextSong|    50975|          119|Showing Out (Get ...|           Mel & Kim|
|1538894186000|       NextSong|    50975|          118|          Speechless|           Lady GaGa|
|1538893922000|       NextSong|    50975|          117|     Time To Pretend|                MGMT|
|1538893631000|     

In [58]:
df.filter('userId = 1333174').select([smin('ts').alias('min_ts'), smax('ts').alias('max_ts')]).show()

+-------------+-------------+
|       min_ts|       max_ts|
+-------------+-------------+
|1538361989000|1538895218000|
+-------------+-------------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.